In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

import os
os.chdir('/content/drive/My Drive/Colab Notebooks/LayoutQT')
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

Mounted at /content/drive/


In [ ]:
!pip install transformers

In [ ]:
import torch

# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [ ]:
def process_data_for_bert(data):

    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

    X_tokenized = tokenizer(list(data['text']), padding=True, truncation=True, max_length=512)
    y = list(data['label'])
    
    dataset = Dataset(X_tokenized, y)

    return dataset

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from transformers import AutoModelForSequenceClassification
from transformers import AutoModelForMaskedLM
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

def runBert(df_train_dataset, df_val_dataset, df_test_dataset):
    
    batch_size=16

    model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=16)
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=False )
     
    train_dataset = process_data_for_bert(df_train_dataset)
    val_dataset = process_data_for_bert(df_val_dataset)
    test_dataset = process_data_for_bert(df_test_dataset)

    # ----- 2. Fine-tune pretrained model -----#
    # Define Trainer parameters
    def compute_metrics(p):
        
        pred, labels = p
        pred = np.argmax(pred, axis=1)

        accuracy = accuracy_score(y_true=labels, y_pred=pred)
        recall = recall_score(y_true=labels, y_pred=pred, average='micro')
        precision = precision_score(y_true=labels, y_pred=pred, average='micro')
        f1 = f1_score(y_true=labels, y_pred=pred, average='micro')

        return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

    # Define Trainer
    args = TrainingArguments(
        "./model/layoutqt_rvlcdip",
        evaluation_strategy = "epoch",
        save_strategy = "epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=10,
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model='f1'
      # optim="adamw_torch"
    )

    def model_init():
        return model

    trainer = Trainer(
        model_init=model_init,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    # New code - wrap collator in a dictionary
    #data_collator = trainer.data_collator
    #trainer.data_collator = lambda data: dict(data_collator(data))
    # End new code

    trainer.train()

    # Make prediction
    raw_pred, _, _ = trainer.predict(test_dataset)

    # Preprocess raw predictions
    y_pred = np.argmax(raw_pred, axis=1)

    return y_pred
    #return roc_auc_score(df_test_dataset['class'], y_pred), f1_score(df_test_dataset['class'], y_pred, average='macro')

In [ ]:
import numpy as np
import pandas as pd

             
df_train = pd.read_csv("./input/rvlcdip/train.csv")
#df_train = df_train[0:10000] 

df_val = pd.read_csv("./input/rvlcdip/val.csv")
#df_val = df_val[0:2000]

df_test = pd.read_csv("./input/rvlcdip/test.csv")
#df_test = df_test[0:2000]

df_train['text'] = df_train['text'].apply(lambda x: str(x))
df_test['text'] = df_test['text'].apply(lambda x: str(x))

df_train['label'] = df_train['label'].apply(lambda x: int(x))
df_test['label'] = df_test['label'].apply(lambda x: int(x))

y_pred = runBert(df_train, df_val, df_test)


In [ ]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import roc_auc_score

print("Accuracy:", accuracy_score(df_test['label'], y_pred))
print("F1:", f1_score(df_test['label'], y_pred, average='micro'))


In [ ]:
print(classification_report(df_test['label'], y_pred))